# Notebook to calculate amplitudes in presence of artificial nonlinearity

## The artificial nonlinearity is introduced via x(t)=max(0,x(t)), i.e. always non-negative

In [1]:
import numpy as np
from math import exp,sqrt,log
import numpy.random
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
x1start=0.2
x2start=0.2
ystart=0.4

In [3]:
##############################################################################################
# Runge_Kutta 4th-order
##############################################################################################
def rk4(F,t,y,ht):
    K0 = ht*F(t,y,dfct)
    K1 = ht*F(t + ht/2.0, y + K0/2.0,dfct)
    K2 = ht*F(t + ht/2.0, y + K1/2.0,dfct)
    K3 = ht*F(t + ht, y + K2,dfct)
    return (K0 + 2.0*K1 + 2.0*K2 + K3)/6.0

####################################################
# Predictor-corrector integration routine: Heun for stochastic differential equations
####################################################
# time t
# time step ht
# number of equations (=dimensionality) n
# random numbers xi
# deterministic and stochastic contributons Fdet and Frand
def Heun(Fdet, Frand,t,y,ht):
    fd1 = np.zeros(3)
    fd2 = np.zeros(3)
    fr = np.zeros(3)
    yt = np.zeros(3)
    fd1 = Fdet(t,y,dfct)
    fr = Frand()
    yt = y + ht*fd1 + fr*sqrt(ht)
    fd2 = Fdet(t+ht,yt,dfct)
    ht2 = ht/2.0
    return ht2*(fd1+fd2) + fr*sqrt(ht)

#activation function
def actfct(z,g,b):
    return 1/(1+exp(-g*(z-b)))

#differential equations motivations
#determnistic part

#nonlinear model
#use this model for Figs 2A-F and Figs 4B and 4C (and comment linear model below)
def Fdet(t,y,dfct):
    Fdet = np.zeros(3)
    Fdet[0] = -k*y[0] + beta*r*actfct(y[0],g1,b1) - beta*actfct(y[2],g2,b2) + q*dfct[0]
    Fdet[1] = -k*y[1] + beta*r*actfct(y[1],g1,b1) - beta*actfct(y[2],g2,b2) + q*dfct[1]
    Fdet[2] = -kinh*y[2] + w_exc*(actfct(y[0],g1,b1) + actfct(y[1],g1,b1))
    return Fdet

'''  
#linear model
# use this model for Figs 4E and F (and comment nonlinear model above)
def Fdet(t,y,dfct):
    Fdet = np.zeros(3)
    Fdet[0] = (r*beta*g1/4-k)*y[0] - beta*g2/4*y[2] + q*dfct[0] + phi
    Fdet[1] = (r*beta*g1/4-k)*y[1] - beta*g2/4*y[2] + q*dfct[1] + phi
    Fdet[2] = -kinh*y[2] + w_exc*(1 - g1*b1/2 + g1/4*(y[0] + y[1]))
    return Fdet
'''

def FrandDet():
    FrandDet = np.zeros(3)
    return FrandDet



# stochastic part of RHS   
def Frand():
    Frand = np.zeros(3)
    xRand0 = rand.normal(0,1)
    xRand1 = rand.normal(0,1)
    Frand[0] = sigma*xRand0
    Frand[1] = sigma*xRand1
    Frand[2] = 0
    return Frand

# model parameters
k = 0.8 # leak excitatory unit
kinh = 0.8 # leak inhibitory unit
w_exc = 3 # excitation strength inhibitory unit
q = 0.1 # frequency of integration
g1 = 10 # gain excitation function
g2 = 10 # gain inhibition function
b1 = 0.5 # midpoint excitation function
b2 = 0.5 # midpoint inhibition function
#beta = 3 #3 # inhibition strength
r = 1


eps_offset = 1e-9

d01 = 7.5
d02 = 7.5
dfct = np.array([d01, d02])
#dfct_start = 7.5
tend = 250  #terminal time 
ht = 0.005  #time step
sigma = 0


maxAmp=[]
minAmp=[]
lastX=[]
paramList=[]
paramRange = [0.01*jj for jj in range(502)]
#paramRange = [1.86+0.1*jj for jj in range(32)]
#paramRange = [1.86+0.02*jj for jj in range(158)]
#paramRange = [3.36+0.02*jj for jj in range(84)]
for param in paramRange:
    #r = param
    beta=param
    paramList.append(param)
    y1 = np.array([x1start, x2start, ystart])  #initial conditions
    #y1 = np.array([0.0, 0.0, 0.0])  #initial conditions
    t = 0.0  #start time
    time = []  
    Ysol1 = []
    #Ysol2 = []
    time.append(t)
    Ysol1.append(y1)
    #Ysol2.append(y2)
    while t <= tend:
        #ht = min(ht,tend-t)
        y1 = y1 + Heun(Fdet,FrandDet,t,y1,ht)
        for ii in range(len(y1)):
            y1[ii] = max(0, y1[ii])
        #y2 = y2 + Heun(Fdet,FrandDet,t,y2,ht)
        t = t + ht
        time.append(t)
        Ysol1.append(y1)
        #Ysol2.append(y2)
    time2=np.asarray(time)
    time3=time2[time2>tend-50]
    time3
    YsolB=np.asarray(Ysol1)
    YsolC=YsolB[time2>tend-50]
    dYsol=[YsolC[jj][0]-YsolC[jj][1] for jj in range(len(YsolC))]
    maxYsol=np.max(dYsol)
    minYsol=np.min(dYsol)
    maxAmp.append(maxYsol)
    minAmp.append(minYsol)
    lastX.append(YsolC[-1])

In [4]:
with open('Amplitudes_beta_X1start_'+ str(round(x1start,2))+'_X2start_'+str(round(x2start,2)) +'_Ystart_'+ str(round(ystart,2)) +'.csv', 'w') as outfile:
    outfile.write("# bifParam, maxAmp, minAmp, x1_last, x2_last, y_last\n")
    for jj in range(len(paramList)):
        outfile.write(str(paramList[jj]) + ',' + str(maxAmp[jj]) + ',' + str(minAmp[jj]) 
                      + ',' + str(lastX[jj][0]) + ',' + str(lastX[jj][1]) + ',' + str(lastX[jj][2]) + '\n')

## Plot of  x1(t)  with artificial nonlinearity

In [ ]:
fig=plt.figure(figsize=(8,6))
plt.plot(np.asarray(time),np.asarray(Ysol1)[:,0],c = 'g',lw = 3,label = r'$x_1$ non-neg')
#plt.plot(np.asarray(time),np.asarray(Ysol2)[:,0],c = 'b', ls = '--', lw = 3,label = r'$x_1$ neg')
plt.legend(loc='lower right', frameon=True, handlelength=4, borderpad=0.4, labelspacing=0.2, fontsize=20)
ax = plt.axes()
plt.xlabel(r'time t ', fontsize=26)
plt.ylabel(r'motivations', fontsize=26)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(22) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(22)
#plt.tick_params(labelsize=14)
#plt.xlim(-200,0)
#plt.ylim(0,0.6)
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.tick_params('both', length=8, width=2, which='major')
ax.tick_params('both', length=6, width=1, which='minor')

#ax.annotate(r'initial motivations', xy=(0, np.asarray(Ysol1)[-1,0]), xytext=(-80, 2.4), fontsize=24,
#            arrowprops=dict(facecolor='k', edgecolor='k', shrink=0.03),
#            horizontalalignment='center', verticalalignment='bottom')

plt.tight_layout();

## Plot of  x2(t)  with artificial nonlinearity

In [ ]:
fig2=plt.figure(figsize=(8,6))
plt.plot(np.asarray(time),np.asarray(Ysol1)[:,1],c = 'g',lw = 3,label = r'$x_2$ non-neg')
#plt.plot(np.asarray(time),np.asarray(Ysol2)[:,1],c = 'b', ls = '--', lw = 3,label = r'$x_2$ neg')
plt.legend(loc='lower right', frameon=True, handlelength=4, borderpad=0.4, labelspacing=0.2, fontsize=20)
ax = plt.axes()
plt.xlabel(r'time t ', fontsize=26)
plt.ylabel(r'motivations', fontsize=26)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(22) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(22)
#plt.tick_params(labelsize=14)
#plt.xlim(-200,0)
#plt.ylim(0,0.6)
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.tick_params('both', length=8, width=2, which='major')
ax.tick_params('both', length=6, width=1, which='minor')

#ax.annotate(r'initial motivations', xy=(0, np.asarray(Ysol1)[-1,0]), xytext=(-80, 2.4), fontsize=24,
#            arrowprops=dict(facecolor='k', edgecolor='k', shrink=0.03),
#            horizontalalignment='center', verticalalignment='bottom')

plt.tight_layout();

## Plot of  x1(t)-x2(t)  with and without artificial nonlinearity

In [ ]:
fig3=plt.figure(figsize=(8,6))
plt.plot(np.asarray(time),np.asarray(Ysol1)[:,0]-np.asarray(Ysol1)[:,1],c = 'g',lw = 3,label = r'$x_1-x_2$')
#plt.plot(np.asarray(time),np.asarray(Ysol2)[:,0]-np.asarray(Ysol2)[:,1],c = 'b', ls = '--', lw = 3,label = r'$x_2$ neg')
plt.legend(loc='lower right', frameon=True, handlelength=4, borderpad=0.4, labelspacing=0.2, fontsize=20)
ax = plt.axes()
plt.xlabel(r'time t ', fontsize=26)
plt.ylabel(r'motivations', fontsize=26)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(22) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(22)
#plt.tick_params(labelsize=14)
#plt.xlim(-200,0)
#plt.ylim(0,0.6)
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.tick_params('both', length=8, width=2, which='major')
ax.tick_params('both', length=6, width=1, which='minor')

#ax.annotate(r'initial motivations', xy=(0, np.asarray(Ysol1)[-1,0]), xytext=(-80, 2.4), fontsize=24,
#            arrowprops=dict(facecolor='k', edgecolor='k', shrink=0.03),
#            horizontalalignment='center', verticalalignment='bottom')

plt.tight_layout();

## Plot of  y(t)  with artificial nonlinearity

In [ ]:
fig4=plt.figure(figsize=(8,6))
plt.plot(np.asarray(time),np.asarray(Ysol1)[:,2],c = 'g',lw = 3,label = r'$y$ non-neg')
#plt.plot(np.asarray(time),np.asarray(Ysol2)[:,2],c = 'b', ls = '--', lw = 3,label = r'$y$ neg')
plt.legend(loc='lower right', frameon=True, handlelength=4, borderpad=0.4, labelspacing=0.2, fontsize=20)
ax = plt.axes()
plt.xlabel(r'time t ', fontsize=26)
plt.ylabel(r'motivations', fontsize=26)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(22) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(22)
#plt.tick_params(labelsize=14)
#plt.xlim(-200,0)
#plt.ylim(0,0.6)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
#ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
ax.tick_params('both', length=8, width=2, which='major')
ax.tick_params('both', length=6, width=1, which='minor')

#ax.annotate(r'initial motivations', xy=(0, np.asarray(Ysol1)[-1,0]), xytext=(-80, 2.4), fontsize=24,
#            arrowprops=dict(facecolor='k', edgecolor='k', shrink=0.03),
#            horizontalalignment='center', verticalalignment='bottom')

plt.tight_layout();